# Asynchronous computing and parallel algorithms

* Asynchronous programming using std::async and std::future
* Asynchronous programming using std::async and std::future
* Experimental support of the parallel algorithms in GCC and MSVC
* Full support of parallel algorithms in HPX

## Asynchronous programming using std::async and std::future

In [1]:
%%writefile example.cpp
#include <future>
#include <iostream>

int add(int a, int b)
{
    return a + b;
}

int main(void)
{
int a = 5;
int b = 5;
std::future<int> result = std::async(add,a,b);

std::cout<< result.get() << std::endl;

return EXIT_SUCCESS;
}

file written: example.cpp

### Compilation

We need to add `-phtread` for the asyncronous execution

In [2]:
!g++ example.cpp -pthread -o example

input_line_3:2:6: error: expected ';' after expression
 !g++ example.cpp -pthread -o example
     ^
     ;
input_line_3:2:3: error: use of undeclared identifier 'g'
 !g++ example.cpp -pthread -o example
  ^
input_line_3:2:7: error: use of undeclared identifier 'example'
 !g++ example.cpp -pthread -o example
      ^
input_line_3:2:20: error: use of undeclared identifier 'pthread'
 !g++ example.cpp -pthread -o example
                   ^
input_line_3:2:29: error: use of undeclared identifier 'o'
 !g++ example.cpp -pthread -o example
                            ^


### Exectution

In [3]:
!./example

input_line_3:2:3: error: expected expression
 !./example
  ^


## Asynchronous programming using hpx::async and hpx::future

In [4]:
#include<run_hpx.cpp>
#include <hpx/future.hpp>
#include <iostream>

In [5]:
int add(int a, int b)
{
    return a + b;
}

In [6]:
run_hpx([](){

hpx::lcos::future<int> result = hpx::async(add,5,5);

std::cout << result.get() ;

});

10

(void) @0x7f5f3a6d8e60


### Example Numerical integration


For this example the Taylor series for the $\sin(x)$
function is computed. The Taylor series is given by,

$$ \sin(x) \approx = \sum\limits_{n=0}^N (-1)^{n-1} \frac{x^{2n}}{(2n)!}.$$

For the concurrent computation, the interval $[0, N]$ is split in two
partitions from $[0, N/2]$ and $[(N/2)+1, N]$, and these are computed
asynchronously using `hpx::async`. Note that each asynchronous function call
returns an `hpx::future` which is needed to synchronize the collection
of the partial results. 

In [7]:
#define long unsigned long long int 

In [8]:
long fact(long n) {
   if (n == 0 || n == 1)
   return 1;
   else
   return n * fact(n - 1);
}



In [9]:
#include <cmath>

// Define the partial taylor function
double taylor(size_t begin, size_t end, size_t n, double x)
{
 double denom = fact(2 * n);
 double res = 0;
 for (size_t i = begin; i != end; ++i)
 {
 res += std::pow(-1, i - 1) * std::pow(x, 2 * n) / denom;
 }
 return res;
}

In [10]:
run_hpx([](){

// Compute the Talor series sin(2.0) for 100 iterations
size_t n = 25;

// Launch two concurrent computations of each partial result
hpx::future<double> f1 = hpx::async(taylor, 0, n / 2, n, 2.);
hpx::future<double> f2 = hpx::async(taylor, (n / 2) + 1, n, n, 2.);

// Introduce a barrier to gather the results
double res = f1.get() + f2.get();

// Print the result
std::cout << "Sin(2.) = " << res << std::endl;

});

Sin(2.) = 0.000148259


(void) @0x7f5f3a6d8e60
